# Training and deploying a tabular model using Vertex custom training job - Part 2

![Training pipeline](../images/custom-tabular.png)

In [13]:
import os
import pprint
import pandas as pd
import tensorflow as tf
import time

import matplotlib.pyplot as plt

from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform_v1beta1 import types
from google.cloud import bigquery
from google.cloud import exceptions

## Configure GCP settings

*Before running the notebook make sure to follow the repo's README file to install the pre-requisites.*

In [14]:
PROJECT = 'jk-vertex-workshop'
REGION = 'us-central1'
PREFIX = 'jk1'

STAGING_BUCKET = f'gs://{PREFIX}-bucket'
VERTEX_SA = f'training-sa@{PROJECT}.iam.gserviceaccount.com'

TENSORBOARD = 'projects/910094146258/locations/us-central1/tensorboards/6406405654306619392'

BQ_DATASET_NAME = f'{PREFIX}_dataset' 
BQ_TRAIN_SPLIT_NAME = 'training'
BQ_VALID_SPLIT_NAME = 'validation'
BQ_TEST_SPLIT_NAME = 'testing'
BQ_LOCATION = 'US'

## Deploying a model to Vertex AI

### Initialize Vertex AI SDK

In [15]:
vertex_ai.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=STAGING_BUCKET
)

### List custom jobs

In [33]:
search_filter = 'display_name="{}_CUSTOM_SCRIPT"'.format(PREFIX)

for job in vertex_ai.CustomJob.list(filter=search_filter, order_by='create_timedesc'):
    print(job.display_name, job.resource_name, job.state, job.create_time)

jk1_CUSTOM_SCRIPT projects/910094146258/locations/us-central1/customJobs/2295819861214887936 JobState.JOB_STATE_RUNNING 2021-06-22 03:20:01.916193+00:00
jk1_CUSTOM_SCRIPT projects/910094146258/locations/us-central1/customJobs/8617184888182800384 JobState.JOB_STATE_RUNNING 2021-06-22 03:09:45.985600+00:00


### Wait for the oldest custom job to complete

In [34]:
while True:
    job_state = job.state
    if job_state != vertex_ai.gapic.JobState.JOB_STATE_SUCCEEDED:
        print("Job has not completed:", job_state)
        if (job_state == vertex_ai.gapic.JobState.JOB_STATE_FAILED or 
           job_state == vertex_ai.gapic.JobState.JOB_STATE_CANCELLED):
            break
    else:
        print("Job has completed")
        break
    time.sleep(60)

Job has not completed: JobState.JOB_STATE_RUNNING
Job has not completed: JobState.JOB_STATE_RUNNING
Job has not completed: JobState.JOB_STATE_RUNNING
Job has not completed: JobState.JOB_STATE_RUNNING
Job has completed


### Retrieve model artifacts

In [39]:
saved_model_path = '{}/model'.format(job.job_spec.base_output_directory.output_uri_prefix)

In [40]:
!gsutil ls {saved_model_path}

gs://jk1-bucket/jobs/jk1_CUSTOM_SCRIPT/20210622_030944/model/
gs://jk1-bucket/jobs/jk1_CUSTOM_SCRIPT/20210622_030944/model/saved_model.pb
gs://jk1-bucket/jobs/jk1_CUSTOM_SCRIPT/20210622_030944/model/assets/
gs://jk1-bucket/jobs/jk1_CUSTOM_SCRIPT/20210622_030944/model/variables/


### Inspect the model

In [41]:
!saved_model_cli show --dir {saved_model_path} --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['dropoff_grid'] tensor_info:
      dtype: DT_STRING
      shape: (-1, 1)
      name: serving_default_dropoff_grid:0
  inputs['euclidean'] tensor_info:
      dtype: DT_DOUBLE
      shape: (-1, 1)
      name: serving_default_euclidean:0
  inputs['payment_type'] tensor_info:
      dtype: DT_STRING
      shape: (-1, 1)
      name: serving_default_payment_type:0
  inputs['pickup_grid'] tensor_info:
      dtype: DT_STRING
      shape: (-1, 1)
      name: serving_default_pickup_grid:0
  inputs['trip_day'] tensor_info:
      dtype: DT_INT64
      shape: (-1, 1)
      name: serving_default_trip_day:0
  inputs['trip_day_of_week'] tensor_info:
      dtype: DT_INT64
      shape: (-1, 1)
      name: serving_default_trip_day_of_week:0
  inputs['trip_hour'] tensor_info:
      dtype: DT_INT64
      shape: (-1, 1)
      name: serving_default_trip_hour:0
  inputs['trip_miles'] tensor_info:
      dtype: DT_DOUBLE
      shape: (-1

### Upload the model

In [42]:
display_name = f'{PREFIX} Chicago taxi tip classifier'
description = 'Chicago taxi tip TensorFlow classifier'
serving_image_uri = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-4:latest'

model = vertex_ai.Model.upload(
    display_name=display_name,
    description=description,
    artifact_uri=saved_model_path,
    serving_container_image_uri=serving_image_uri
)

model.wait()

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/910094146258/locations/us-central1/models/7226878823139115008/operations/8300640163470508032
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/910094146258/locations/us-central1/models/7226878823139115008
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/910094146258/locations/us-central1/models/7226878823139115008')


### Create an endpoint

In [43]:
display_name = f'{PREFIX} Taxi tip classifier endpoint'

endpoint = vertex_ai.Endpoint.create(display_name=display_name)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/910094146258/locations/us-central1/endpoints/4297331245999718400/operations/1211974349989347328
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/910094146258/locations/us-central1/endpoints/4297331245999718400
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/910094146258/locations/us-central1/endpoints/4297331245999718400')


### Deploy the Model

In [44]:
deployed_model_display_name = f'{PREFIX}-taxi-v1'
traffic_percentage = 100
machine_type = 'n1-standard-4'

endpoint = model.deploy(
        endpoint=endpoint,
        deployed_model_display_name=deployed_model_display_name,
        machine_type=machine_type,
        traffic_percentage=traffic_percentage)


INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/910094146258/locations/us-central1/endpoints/4297331245999718400
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/910094146258/locations/us-central1/endpoints/4297331245999718400/operations/1455168729867354112
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/910094146258/locations/us-central1/endpoints/4297331245999718400


## Invoking the deployed model using Vertex SDK

### Get the endpoint

In [45]:
filter = f'display_name="{PREFIX} Taxi tip classifier endpoint"'

for endpoint_info in vertex_ai.Endpoint.list(filter=filter):
    print(endpoint_info)
    
endpoint = vertex_ai.Endpoint(endpoint_info.resource_name)

resource name: projects/910094146258/locations/us-central1/endpoints/4297331245999718400


### Call the endpoint

In [46]:
test_instances = [  
    
    {
        "dropoff_grid": ["POINT(-87.6 41.9)"],
        "euclidean": [2064.2696],
        "payment_type": ["Credit Card"],
        "pickup_grid": ["POINT(-87.6 41.9)"],
        "trip_miles": [1.37],
        "trip_day": [12],
        "trip_hour": [16],
        "trip_month": [2],
        "trip_day_of_week": [4],
        "trip_seconds": [555]
    }
]

predictions = endpoint.predict(instances=test_instances)
prob = tf.nn.sigmoid(predictions[0])
print('Probability of tip > 20%:', prob.numpy())

Probability of tip > 20%: [[0.75484097]]


## Clean up

### Undeploy models

In [47]:
endpoint.list_models()

[id: "7622184039650164736"
model: "projects/910094146258/locations/us-central1/models/7226878823139115008"
display_name: "jk1-taxi-v1"
create_time {
  seconds: 1624333147
  nanos: 505658000
}
dedicated_resources {
  machine_spec {
    machine_type: "n1-standard-4"
  }
  min_replica_count: 1
  max_replica_count: 1
}
]

In [48]:
endpoint.undeploy_all()

INFO:google.cloud.aiplatform.models:Undeploying Endpoint model: projects/910094146258/locations/us-central1/endpoints/4297331245999718400
INFO:google.cloud.aiplatform.models:Undeploy Endpoint model backing LRO: projects/910094146258/locations/us-central1/endpoints/4297331245999718400/operations/4164083905730707456
INFO:google.cloud.aiplatform.models:Endpoint model undeployed. Resource name: projects/910094146258/locations/us-central1/endpoints/4297331245999718400


resource name: projects/910094146258/locations/us-central1/endpoints/4297331245999718400

### Delete endpoint

In [49]:
endpoint.delete()

INFO:google.cloud.aiplatform.base:Deleting Endpoint : projects/910094146258/locations/us-central1/endpoints/4297331245999718400
INFO:google.cloud.aiplatform.base:Delete Endpoint  backing LRO: projects/910094146258/locations/us-central1/operations/2554047038945755136
INFO:google.cloud.aiplatform.base:Endpoint deleted. . Resource name: projects/910094146258/locations/us-central1/endpoints/4297331245999718400


### Delete model

In [50]:
model.delete()

INFO:google.cloud.aiplatform.base:Deleting Model : projects/910094146258/locations/us-central1/models/7226878823139115008
INFO:google.cloud.aiplatform.base:Delete Model  backing LRO: projects/910094146258/locations/us-central1/operations/3610141151564136448
INFO:google.cloud.aiplatform.base:Model deleted. . Resource name: projects/910094146258/locations/us-central1/models/7226878823139115008
